In [1]:
# aircraft_forecasting_optuna.py

# Importaciones básicas
import sys
import os
from pathlib import Path
import logging
import warnings
warnings.filterwarnings('ignore')

# Añadir el directorio padre al path
sys.path.append(str(Path().cwd().parent))

# Importar bibliotecas de análisis
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)

# Importar módulos personalizados
from models import (
    ModelConfig,
    ATCAircraftDataLoader,
    AircraftDataPreprocessor,
    AircraftFeatureEngineer,
    ARIMAModel,
    ProphetModel,
    RandomForestModel,
    LSTMModel,
    EnsembleModel,
    AircraftForecaster
)

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuración
config = ModelConfig()
RANDOM_STATE = 42

In [3]:
def load_and_prepare_data(forecast_horizon=7):
    """Carga y prepara los datos para el entrenamiento."""
    logger.info("Cargando y preparando datos...")
    
    # 1. Cargar datos
    data_loader = ATCAircraftDataLoader(config)
    df = data_loader.get_training_data('daily_atc')
    
    # 2. Preprocesar datos
    preprocessor = AircraftDataPreprocessor(config)
    df_processed = preprocessor.preprocess_daily_data(df)
    
    # 3. Ingeniería de características
    feature_engineer = AircraftFeatureEngineer(config)
    df_featured = feature_engineer.create_features(df_processed)
    df_featured = feature_engineer.create_lagged_target(
        df_featured, 
        forecast_horizon=forecast_horizon
    )
    
    # 4. Preparar datos para modelado
    X, y = feature_engineer.select_features_for_model(df_featured)
    
    logger.info(f"Datos preparados: {len(X)} muestras, {len(X.columns)} características")
    
    return X, y, df_featured


In [5]:
def objective(trial, X, y):
    """
    Función objetivo para la optimización con Optuna.
    
    Args:
        trial: Objeto de prueba de Optuna
        X: Características de entrenamiento
        y: Variable objetivo
        
    Returns:
        Error de validación (MAE) a minimizar
    """
    # Selección del modelo
    algorithm = trial.suggest_categorical('algorithm', ['random_forest', 'prophet', 'lstm', 'arima'])

    # Crear una copia de la configuración
    trial_config = ModelConfig()

    if algorithm == 'random_forest':
        # Espacio de búsqueda para Random Forest
        trial_config.models['random_forest'] = {
            'n_estimators': trial.suggest_int('rf_n_estimators', 50, 500, step=50),
            'max_depth': trial.suggest_int('rf_max_depth', 3, 30),
            'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('rf_min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('rf_max_features', ['sqrt', 'log2', None]),
            'bootstrap': trial.suggest_categorical('rf_bootstrap', [True, False]),
            'random_state': RANDOM_STATE
        }
        
        model = RandomForestModel(trial_config)
        
    elif algorithm == 'prophet':
        # Espacio de búsqueda para Prophet
        trial_config.models['prophet'] = {
            'yearly_seasonality': trial.suggest_categorical('prophet_yearly', [True, False]),
            'weekly_seasonality': trial.suggest_categorical('prophet_weekly', [True, False]),
            'daily_seasonality': trial.suggest_categorical('prophet_daily', [True, False]),
            'changepoint_prior_scale': trial.suggest_float('prophet_changepoint_prior_scale', 0.001, 0.5, log=True),
            'seasonality_prior_scale': trial.suggest_float('prophet_seasonality_prior_scale', 0.1, 10, log=True),
            'seasonality_mode': trial.suggest_categorical('prophet_seasonality_mode', ['additive', 'multiplicative']),
            'changepoint_range': trial.suggest_float('prophet_changepoint_range', 0.8, 0.95),
            'n_changepoints': trial.suggest_int('prophet_n_changepoints', 10, 50, step=5)
        }
        
        model = ProphetModel(trial_config)
        
    elif algorithm == 'lstm':
        # Espacio de búsqueda para LSTM
        trial_config.models['lstm'] = {
            'sequence_length': trial.suggest_int('lstm_sequence_length', 7, 30, step=7),
            'hidden_units': trial.suggest_int('lstm_hidden_units', 32, 256, step=32),
            'dropout_rate': trial.suggest_float('lstm_dropout', 0.1, 0.5, step=0.1),
            'epochs': trial.suggest_int('lstm_epochs', 50, 200, step=50),
            'batch_size': trial.suggest_categorical('lstm_batch_size', [16, 32, 64]),
            'learning_rate': trial.suggest_float('lstm_learning_rate', 1e-4, 1e-2, log=True),
            'optimizer': trial.suggest_categorical('lstm_optimizer', ['adam', 'rmsprop']),
            'num_layers': trial.suggest_int('lstm_num_layers', 1, 3)
        }
        
        model = LSTMModel(trial_config)
        
    elif algorithm == 'arima':
        # Espacio de búsqueda para ARIMA
        p = trial.suggest_int('arima_p', 0, 5)
        d = trial.suggest_int('arima_d', 0, 2)
        q = trial.suggest_int('arima_q', 0, 5)
        P = trial.suggest_int('arima_P', 0, 3)
        D = trial.suggest_int('arima_D', 0, 2)
        Q = trial.suggest_int('arima_Q', 0, 3)
        s = 7  # Estacionalidad semanal
        
        trial_config.models['arima'] = {
            'order': (p, d, q),
            'seasonal_order': (P, D, Q, s)
        }
        
        model = ARIMAModel(trial_config)
    
    else:
        raise ValueError(f"Tipo de modelo no soportado: {algorithm}")
    
    # Validación cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Ajustar el modelo
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_val)

        # Alinear predicciones con el target y filtrar valores no finitos
        y_pred = np.asarray(y_pred).ravel()

        if len(y_pred) != len(y_val):
            min_len = min(len(y_pred), len(y_val))
            y_pred = y_pred[-min_len:]
            y_val_aligned = y_val.iloc[-min_len:]
        else:
            y_val_aligned = y_val

        valid_mask = np.isfinite(y_pred)
        if not valid_mask.any():
            raise ValueError("Predicciones no válidas: todas son NaN o infinitas")

        y_pred = y_pred[valid_mask]
        y_val_aligned = y_val_aligned.iloc[np.where(valid_mask)[0]]

        # Calcular métricas
        mae = mean_absolute_error(y_val_aligned, y_pred)
        scores.append(mae)
    
    # Devolver el MAE promedio
    return np.mean(scores)


In [9]:
def objective_emsemble(trial, X, y):
    """
    Función objetivo para la optimización con Optuna.
    
    Args:
        trial: Objeto de prueba de Optuna
        X: Características de entrenamiento
        y: Variable objetivo
        
    Returns:
        Error de validación (MAE) a minimizar
    """
    # Selección del modelo
    algorithm = trial.suggest_categorical('algorithm', ['random_forest', 'prophet', 'lstm', 'arima', 'ensemble'])

    # Crear una copia de la configuración
    trial_config = ModelConfig()
    
    # Configuración común para los modelos base
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 50, 500, step=50),
        'max_depth': trial.suggest_int('rf_max_depth', 3, 30),
        'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('rf_min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('rf_max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('rf_bootstrap', [True, False]),
        'random_state': RANDOM_STATE
    }
    
    prophet_params = {
        'yearly_seasonality': trial.suggest_categorical('prophet_yearly', [True, False]),
        'weekly_seasonality': trial.suggest_categorical('prophet_weekly', [True, False]),
        'daily_seasonality': trial.suggest_categorical('prophet_daily', [True, False]),
        'changepoint_prior_scale': trial.suggest_float('prophet_changepoint_prior_scale', 0.001, 0.5, log=True),
        'seasonality_prior_scale': trial.suggest_float('prophet_seasonality_prior_scale', 0.1, 10, log=True),
        'seasonality_mode': trial.suggest_categorical('prophet_seasonality_mode', ['additive', 'multiplicative']),
        'changepoint_range': trial.suggest_float('prophet_changepoint_range', 0.8, 0.95),
        'n_changepoints': trial.suggest_int('prophet_n_changepoints', 10, 50, step=5)
    }
    
    lstm_params = {
        'sequence_length': trial.suggest_int('lstm_sequence_length', 7, 30, step=7),
        'hidden_units': trial.suggest_int('lstm_hidden_units', 32, 256, step=32),
        'dropout_rate': trial.suggest_float('lstm_dropout', 0.1, 0.5, step=0.1),
        'epochs': trial.suggest_int('lstm_epochs', 50, 200, step=50),
        'batch_size': trial.suggest_categorical('lstm_batch_size', [16, 32, 64]),
        'learning_rate': trial.suggest_float('lstm_learning_rate', 1e-4, 1e-2, log=True),
        'optimizer': trial.suggest_categorical('lstm_optimizer', ['adam', 'rmsprop']),
        'num_layers': trial.suggest_int('lstm_num_layers', 1, 3)
    }
    
    # Parámetros ARIMA
    p = trial.suggest_int('arima_p', 0, 5)
    d = trial.suggest_int('arima_d', 0, 2)
    q = trial.suggest_int('arima_q', 0, 5)
    P = trial.suggest_int('arima_P', 0, 3)
    D = trial.suggest_int('arima_D', 0, 2)
    Q = trial.suggest_int('arima_Q', 0, 3)
    s = 7  # Estacionalidad semanal

    if algorithm == 'random_forest':
        trial_config.models['random_forest'] = rf_params
        model = RandomForestModel(trial_config)
        
    elif algorithm == 'prophet':
        trial_config.models['prophet'] = prophet_params
        model = ProphetModel(trial_config)
        
    elif algorithm == 'lstm':
        trial_config.models['lstm'] = lstm_params
        model = LSTMModel(trial_config)
        
    elif algorithm == 'arima':
        trial_config.models['arima'] = {
            'order': (p, d, q),
            'seasonal_order': (P, D, Q, s)
        }
        model = ARIMAModel(trial_config)
        
    elif algorithm == 'ensemble':
        # Crear configuración para el ensemble
        trial_config = ModelConfig()
        
        # Definir espacio de búsqueda para los pesos
        weights = {
            # 'arima': trial.suggest_float('ensemble_weight_arima', 0, 1),
            # 'prophet': trial.suggest_float('ensemble_weight_prophet', 0, 1),
            'random_forest': trial.suggest_float('ensemble_weight_rf', 0, 1),
            'lstm': trial.suggest_float('ensemble_weight_lstm', 0, 1)
        }
        
        # Normalizar pesos para que sumen 1
        total = sum(weights.values())
        weights = {k: v/total for k, v in weights.items()}
        
        # Configurar pesos en la configuración
        trial_config.models['ensemble'] = {'weights': weights}
        
        # Configurar parámetros de los modelos base
        trial_config.models['random_forest'] = rf_params
        # trial_config.models['prophet'] = prophet_params
        trial_config.models['lstm'] = lstm_params
        # trial_config.models['arima'] = {
        #     'order': (p, d, q),
        #     'seasonal_order': (P, D, Q, s)
        # }
        
        # Crear el ensemble y añadir modelos base
        ensemble = EnsembleModel(trial_config)
        # ensemble.add_model(ARIMAModel(trial_config), weight=weights['arima'])
        # ensemble.add_model(ProphetModel(trial_config), weight=weights['prophet'])
        ensemble.add_model(RandomForestModel(trial_config), weight=weights['random_forest'])
        ensemble.add_model(LSTMModel(trial_config), weight=weights['lstm'])
        
        model = ensemble
    
    else:
        raise ValueError(f"Tipo de modelo no soportado: {algorithm}")
    
    # Validación cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Ajustar el modelo
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_val)

        # Alinear predicciones con el target y filtrar valores no finitos
        y_pred = np.asarray(y_pred).ravel()

        if len(y_pred) != len(y_val):
            min_len = min(len(y_pred), len(y_val))
            y_pred = y_pred[-min_len:]
            y_val_aligned = y_val.iloc[-min_len:]
        else:
            y_val_aligned = y_val

        valid_mask = np.isfinite(y_pred)
        if not valid_mask.any():
            raise ValueError("Predicciones no válidas: todas son NaN o infinitas")

        y_pred = y_pred[valid_mask]
        y_val_aligned = y_val_aligned.iloc[np.where(valid_mask)[0]]

        # Calcular métricas
        mae = mean_absolute_error(y_val_aligned, y_pred)
        scores.append(mae)
    
    # Devolver el MAE promedio
    return np.mean(scores)


In [7]:
def optimize_hyperparameters(X, y, n_trials=50):
    """
    Optimiza los hiperparámetros usando Optuna.
    
    Args:
        X: Características
        y: Variable objetivo
        n_trials: Número de pruebas a realizar
        
    Returns:
        study: Objeto de estudio de Optuna
    """
    # Crear o cargar estudio con almacenamiento persistente
    try:
        # Intentar cargar un estudio existente
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_name,
            load_if_exists=True,
            direction='minimize',
            sampler=TPESampler(seed=RANDOM_STATE)
        )
        logger.info(f"Estudio cargado. Número de trials existentes: {len(study.trials)}")
    except Exception as e:
        # Si no existe, crear uno nuevo
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_name,
            direction='minimize',
            sampler=TPESampler(seed=RANDOM_STATE)
        )
        logger.info("Nuevo estudio creado")
    
    # Función objetivo parcial
    def objective_wrapper(trial):
        return objective_emsemble(trial, X, y)
    
    # Calcular cuántos trials nuevos necesitamos
    remaining_trials = max(0, n_trials - len(study.trials))
    
    if remaining_trials > 0:
        logger.info(f"Iniciando optimización con {remaining_trials} pruebas nuevas...")
        study.optimize(objective_wrapper, n_trials=remaining_trials, show_progress_bar=True)
    else:
        logger.info(f"Ya se han completado {len(study.trials)} trials. No se necesitan más pruebas.")
    
    # Mostrar resultados
    logger.info("\nResumen de la optimización:")
    logger.info(f"Número total de trials: {len(study.trials)}")
    logger.info(f"Mejor valor (MAE): {study.best_value:.4f}")
    logger.info("Mejores parámetros encontrados:")
    for key, value in study.best_params.items():
        logger.info(f"  {key}: {value}")
    
    return study


In [2]:
# Configuración de almacenamiento para Optuna
import sqlite3
from pathlib import Path

# Crear directorio para almacenamiento si no existe
storage_dir = Path("optuna_storage")
storage_dir.mkdir(exist_ok=True)

# Configurar el almacenamiento
storage_name = f"sqlite:///{storage_dir}/aircraft_forecasting.db"
study_name = "aircraft_forecasting_study"

In [10]:
"""Función principal para ejecutar la optimización."""
# Cargar y preparar datos
X, y, _ = load_and_prepare_data(forecast_horizon=7)

study = optimize_hyperparameters(X, y, n_trials=100)

2026-01-04 13:14:27,440 - __main__ - INFO - Cargando y preparando datos...
2026-01-04 13:14:27,441 - models.data_loader - INFO - Cargando datos diarios ATC desde: data/ATC csvs/atc_dayatcopsummary_202512301506.csv
2026-01-04 13:14:27,449 - models.data_loader - INFO - Datos diarios cargados: 764 registros, columnas: ['arrivals', 'departures', 'overflights', 'nationals', 'unknown', 'total', 'fpp']
2026-01-04 13:14:27,450 - models.data_loader - INFO - Datos de entrenamiento preparados: 764 registros del 2022-12-11 00:00:00 al 2025-12-30 00:00:00
2026-01-04 13:14:27,450 - models.preprocessing - INFO - Iniciando preprocesamiento de datos diarios
2026-01-04 13:14:27,455 - models.preprocessing - INFO - Frecuencia diaria asegurada: 1116 días (original: 764)
2026-01-04 13:14:27,461 - models.preprocessing - INFO - Validación de integridad: OK
2026-01-04 13:14:27,462 - models.preprocessing - INFO - Preprocesamiento completado: 1116 registros
2026-01-04 13:14:27,462 - models.features - INFO - Inic

  0%|          | 0/100 [00:00<?, ?it/s]

2026-01-04 13:14:27,773 - models.model - INFO - Entrenando Prophet
2026-01-04 13:14:27,787 - cmdstanpy - INFO - Chain [1] start processing
2026-01-04 13:14:27,807 - cmdstanpy - INFO - Chain [1] done processing
2026-01-04 13:14:27,809 - models.model - INFO - Prophet entrenado exitosamente
2026-01-04 13:14:27,862 - models.model - INFO - Entrenando Prophet
2026-01-04 13:14:27,876 - cmdstanpy - INFO - Chain [1] start processing
2026-01-04 13:14:27,892 - cmdstanpy - INFO - Chain [1] done processing
2026-01-04 13:14:27,895 - models.model - INFO - Prophet entrenado exitosamente
2026-01-04 13:14:27,946 - models.model - INFO - Entrenando Prophet
2026-01-04 13:14:27,961 - cmdstanpy - INFO - Chain [1] start processing
2026-01-04 13:14:27,992 - cmdstanpy - INFO - Chain [1] done processing
2026-01-04 13:14:27,994 - models.model - INFO - Prophet entrenado exitosamente
2026-01-04 13:14:28,045 - models.model - INFO - Entrenando Prophet
2026-01-04 13:14:28,060 - cmdstanpy - INFO - Chain [1] start proce

[I 2026-01-04 13:14:28,288] Trial 0 finished with value: 455.68154493654646 and parameters: {'algorithm': 'prophet', 'rf_n_estimators': 100, 'rf_max_depth': 4, 'rf_min_samples_split': 18, 'rf_min_samples_leaf': 7, 'rf_max_features': None, 'rf_bootstrap': True, 'prophet_yearly': False, 'prophet_weekly': False, 'prophet_daily': True, 'prophet_changepoint_prior_scale': 0.044809759182149515, 'prophet_seasonality_prior_scale': 0.19010245319870356, 'prophet_seasonality_mode': 'multiplicative', 'prophet_changepoint_range': 0.8684104976325554, 'prophet_n_changepoints': 45, 'lstm_sequence_length': 7, 'lstm_hidden_units': 160, 'lstm_dropout': 0.30000000000000004, 'lstm_epochs': 50, 'lstm_batch_size': 16, 'lstm_learning_rate': 0.007902619549708232, 'lstm_optimizer': 'adam', 'lstm_num_layers': 1, 'arima_p': 0, 'arima_d': 2, 'arima_q': 2, 'arima_P': 0, 'arima_D': 1, 'arima_Q': 0}. Best is trial 0 with value: 455.68154493654646.


2026-01-04 13:14:28,707 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:14:28,758 - models.model - INFO - Entrenando Random Forest con 300 árboles
2026-01-04 13:14:29,062 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:14:29,114 - models.model - INFO - Entrenando Random Forest con 300 árboles
2026-01-04 13:14:29,472 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:14:29,525 - models.model - INFO - Entrenando Random Forest con 300 árboles
2026-01-04 13:14:30,060 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:14:30,112 - models.model - INFO - Entrenando Random Forest con 300 árboles
2026-01-04 13:14:30,637 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:14:30,878 - models.model - INFO - Entrenando LSTM con sequence_length=28


...

[I 2026-01-04 13:45:43,930] Trial 97 finished with value: 211.90064388117503 and parameters: {'algorithm': 'random_forest', 'rf_n_estimators': 350, 'rf_max_depth': 18, 'rf_min_samples_split': 3, 'rf_min_samples_leaf': 2, 'rf_max_features': 'sqrt', 'rf_bootstrap': True, 'prophet_yearly': False, 'prophet_weekly': True, 'prophet_daily': False, 'prophet_changepoint_prior_scale': 0.010034147145266611, 'prophet_seasonality_prior_scale': 1.7112350769282512, 'prophet_seasonality_mode': 'multiplicative', 'prophet_changepoint_range': 0.9153104176962223, 'prophet_n_changepoints': 20, 'lstm_sequence_length': 7, 'lstm_hidden_units': 128, 'lstm_dropout': 0.4, 'lstm_epochs': 150, 'lstm_batch_size': 16, 'lstm_learning_rate': 0.008039449630546454, 'lstm_optimizer': 'adam', 'lstm_num_layers': 2, 'arima_p': 5, 'arima_d': 0, 'arima_q': 2, 'arima_P': 1, 'arima_D': 2, 'arima_Q': 2}. Best is trial 63 with value: 186.1624744723504.


2026-01-04 13:45:44,165 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:45:44,184 - models.model - INFO - Entrenando Random Forest con 100 árboles
2026-01-04 13:45:44,271 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:45:44,289 - models.model - INFO - Entrenando Random Forest con 100 árboles
2026-01-04 13:45:44,391 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:45:44,409 - models.model - INFO - Entrenando Random Forest con 100 árboles
2026-01-04 13:45:44,537 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:45:44,555 - models.model - INFO - Entrenando Random Forest con 100 árboles
2026-01-04 13:45:44,702 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:45:44,901 - models.model - INFO - Entrenando ensemble de modelos
2026-01-04 13:45:44,902 - models.model - INFO - Entrenando random_forest...
2026-01-04 13:45:44,902 - models.model - INFO - Entrenando Rando

[I 2026-01-04 13:45:44,730] Trial 98 finished with value: 189.6147255166758 and parameters: {'algorithm': 'random_forest', 'rf_n_estimators': 100, 'rf_max_depth': 19, 'rf_min_samples_split': 5, 'rf_min_samples_leaf': 9, 'rf_max_features': 'sqrt', 'rf_bootstrap': True, 'prophet_yearly': False, 'prophet_weekly': True, 'prophet_daily': False, 'prophet_changepoint_prior_scale': 0.17061925207067463, 'prophet_seasonality_prior_scale': 2.835409397598875, 'prophet_seasonality_mode': 'multiplicative', 'prophet_changepoint_range': 0.8836878012673354, 'prophet_n_changepoints': 45, 'lstm_sequence_length': 21, 'lstm_hidden_units': 96, 'lstm_dropout': 0.2, 'lstm_epochs': 150, 'lstm_batch_size': 64, 'lstm_learning_rate': 0.006493411197441486, 'lstm_optimizer': 'rmsprop', 'lstm_num_layers': 2, 'arima_p': 3, 'arima_d': 0, 'arima_q': 1, 'arima_P': 1, 'arima_D': 2, 'arima_Q': 1}. Best is trial 63 with value: 186.1624744723504.


2026-01-04 13:45:45,079 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:45:45,096 - models.model - INFO - Entrenando lstm...
2026-01-04 13:45:45,097 - models.model - INFO - Entrenando LSTM con sequence_length=21
2026-01-04 13:45:57,004 - models.model - INFO - LSTM entrenado exitosamente
2026-01-04 13:45:57,005 - models.model - INFO - Ensemble entrenado exitosamente
2026-01-04 13:45:57,401 - models.model - INFO - Entrenando ensemble de modelos
2026-01-04 13:45:57,401 - models.model - INFO - Entrenando random_forest...
2026-01-04 13:45:57,401 - models.model - INFO - Entrenando Random Forest con 250 árboles
2026-01-04 13:45:57,604 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-04 13:45:57,629 - models.model - INFO - Entrenando lstm...
2026-01-04 13:45:57,629 - models.model - INFO - Entrenando LSTM con sequence_length=21
2026-01-04 13:46:15,218 - models.model - INFO - LSTM entrenado exitosamente
2026-01-04 13:46:15,219 - models.model - INFO 

[I 2026-01-04 13:47:45,047] Trial 99 finished with value: 218.33426733812243 and parameters: {'algorithm': 'ensemble', 'rf_n_estimators': 250, 'rf_max_depth': 26, 'rf_min_samples_split': 13, 'rf_min_samples_leaf': 10, 'rf_max_features': 'log2', 'rf_bootstrap': True, 'prophet_yearly': False, 'prophet_weekly': True, 'prophet_daily': False, 'prophet_changepoint_prior_scale': 0.01973333831742301, 'prophet_seasonality_prior_scale': 0.6372246503166865, 'prophet_seasonality_mode': 'multiplicative', 'prophet_changepoint_range': 0.8928093297856822, 'prophet_n_changepoints': 35, 'lstm_sequence_length': 21, 'lstm_hidden_units': 96, 'lstm_dropout': 0.30000000000000004, 'lstm_epochs': 200, 'lstm_batch_size': 64, 'lstm_learning_rate': 0.007228230756391968, 'lstm_optimizer': 'adam', 'lstm_num_layers': 3, 'arima_p': 4, 'arima_d': 0, 'arima_q': 3, 'arima_P': 1, 'arima_D': 2, 'arima_Q': 3, 'ensemble_weight_rf': 0.9779178389553312, 'ensemble_weight_lstm': 0.25290360275854246}. Best is trial 63 with value

In [8]:
# Configuración global de estilo
sns.set_theme(style="whitegrid", context="talk", palette="deep")
plt.rcParams.update(
    {
        "axes.titlesize": 18,
        "axes.labelsize": 14,
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
        "legend.fontsize": 11
    }
)